## Step1 导入相关包

In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from transformers import AutoTokenizer,AutoModelForSequenceClassification

## Step2 加载数据

In [2]:
import pandas as pd
data = pd.read_csv('ChnSentiCorp_htl_all.csv')
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [4]:
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


## Step3 创建Dataset

In [5]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.data = pd.read_csv('./ChnSentiCorp_htl_all.csv')
        self.data = self.data.dropna()

    def __getitem__(self, index):
        data = self.data.iloc[index]
        return data['review'], data['label']

    def __len__(self):
        return len(self.data)

In [6]:
dataset=MyDataset() # 便于后续的DataLoader与数据划分
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [7]:
from torch.utils.data import random_split

trainset,validset = random_split(dataset,lengths=[0.9,0.1])
len(trainset),len(validset)

(6989, 776)

In [8]:
for i in range(10):
    print(trainset[i])

('客房环境级差，房间里时常可以看见“小强”，新铺的床单上由占整个面积3分之2的不知道是什么液体留下的印记，还夹杂着3、4个黑鞋印，恶心之极。卫生间所有的下水都很慢，在浴缸里冲澡每5、6分钟得听一小会儿，要不就水淹七军了，毛巾都是灰色的，实在是差劲。建议想入住德请慎重考虑。', 0)
('如果你有一小段假期去阳朔,在阳朔居住超过3天的，蔽龙阁绝对是个非常理想的居住地点.房间大，干净，就是要注意自备花露水和驱蚊水,带个电的驱蚊香最好了，那里是田园生活么...蚊子..苍蝇比城市多.....在山水环抱中，感觉不是住在宾馆,很舒服...李阿姨很热情...做饭的阿姨也很好..如果你不想出去吃了，那10-20元/人,可以在那里搭伙,看吃什么就给多少.你也可以提出要求，想吃什么,李阿姨会帮你买好做好的.离西街是有距离的，不过我觉的还是很方便，走到村口,有摩托坐,10元到西街,回来也一样....就个人而言,我喜欢租自行车到周遍的小村子里去逛...借这个评论的地方,特别感谢李阿姨和做饭的阿姨，因为我是吃全素的，李阿姨知道后，每天都很注意弄早餐和我搭伙的饮食,住了三天，吃了三天地道的农家新鲜蔬菜...呵呵..走的时候..李阿姨特地送我出门..帮我叫了车还砍价...叫到了桂林-荔蒲的车..结果10元就回了桂林...＾＿＾总之一句话，下次去阳朔．．只住蔽龙阁了补充点评2008年5月6日：特别告示：到旅馆门口的马路修好了，就是村口大概有一点路，还是没修好的，马路更新信息．．２００８年５月６日，方便下次去的各位驴友', 1)
('这是我第二次入住瑞和园了，两次都是住在二楼04的房间。这次还发现房间新添置了一台联想电脑，非常体贴周到！400元一晚的价钱，还OK吧，对于背包客确实有点儿贵了，可对于那些希望静静休假，不被任何繁琐干扰的朋友们，我觉得是值得推荐的！来到丽江，住在瑞和园，每天都只呆在客栈看书喝茶已经值回票价了！下次再游古城，不用带电脑了，我想我还会继续选择瑞和园！', 1)
('临街的房间，特别的吵，晚上几乎无法睡觉。房间隔音不好，白天听到隔壁“嘿咻”的声音，晚上听到隔壁打呼噜，让人无法睡觉。', 0)
('价格适中，总体感觉还可以。地理位置上稍微难找一点（因为不是靠大路边），但离商业街北京路很近，酒店门外就有一条小食街。', 1)
('订的是标A，房间装修比较新。只是洗澡的热水要等个快

## Step5 创建DataLoader

In [9]:
import torch
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    texts,labels = [],[]
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])

    inputs = tokenizer(texts, max_length=128,padding='max_length',truncation=True,return_tensors='pt')
    inputs['labels'] = torch.tensor(labels)
    return inputs


In [10]:
from torch.utils.data import DataLoader
trainloader = DataLoader(trainset,batch_size=32,shuffle=True,collate_fn=collate_func)
valiloader = DataLoader(validset,batch_size=64,shuffle=False,collate_fn=collate_func)

In [11]:
next(enumerate(valiloader))

(0,
 {'input_ids': tensor([[ 101, 6421, 6983,  ...,    0,    0,    0],
         [ 101, 1184, 1126,  ..., 3300, 2990,  102],
         [ 101, 2791, 7313,  ...,    0,    0,    0],
         ...,
         [ 101, 3302, 1218,  ...,  123, 3189,  102],
         [ 101, 3302, 1218,  ...,    0,    0,    0],
         [ 101, 2600,  860,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
         0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 

## Step6 创建模型及优化器

In [12]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model.parameters()

<generator object Module.parameters at 0x7f43e2582900>

In [14]:
optimizer = Adam(model.parameters(),lr=2e-5)

## Step7 训练与验证

In [15]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in valiloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits,dim=-1)
            acc_num += (pred.long() == batch['labels'].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3,log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step ==0:
                print(f"ep:{ep}, global_step:{global_step}, loss:{output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"e:{ep}, acc:{acc}")


## Step8 模型训练

In [16]:
train()

ep:0, global_step:0, loss:1.0386736392974854
ep:0, global_step:100, loss:0.4340834617614746
ep:0, global_step:200, loss:0.12030831724405289
e:0, acc:0.89304119348526
ep:1, global_step:300, loss:0.3341277539730072
ep:1, global_step:400, loss:0.18069854378700256
e:1, acc:0.8981958627700806
ep:2, global_step:500, loss:0.297456830739975
ep:2, global_step:600, loss:0.11136312782764435
e:2, acc:0.905927836894989


## Step9 模型预测

In [17]:
sen = '我觉得这家酒店中等偏差，其中隔音效果不好！'
id2_lable = {0:'差评！',1:'好评！'}
model.eval()
with torch.inference_mode():
    input = tokenizer(sen,return_tensors='pt')
    input = {k:v.cuda() for k,v in input.items()}
    output = model(**input)
    pred = torch.argmax(output.logits,dim=-1)
    print(f"输入：{sen}，模型预测结果：{id2_lable.get(pred.item())}")

输入：我觉得这家酒店中等偏差，其中隔音效果不好！，模型预测结果：差评！


In [18]:
from transformers import pipeline
# model.config.id2label
model.config.id2label = id2_lable
pipe = pipeline("text-classification",model=model,tokenizer=tokenizer,device=0)

In [19]:
pipe(sen) 

[{'label': '差评！', 'score': 0.9226927161216736}]